# 6조 (17011709 정선아, 17011741 문성용, 17011742 김소영)

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

In [2]:
MovieLens_df = pd.read_table("ratings.dat", sep='::', header=None, names=['UserID','MovieID','Rating','Timestamp'])
MovieLens_df.drop(['Timestamp'], axis=1, inplace=True)
MovieLens_pivot = MovieLens_df.pivot_table(index='UserID', columns='MovieID', values='Rating')
row_index = MovieLens_pivot.isnull().sum(axis = 1).sort_values() # NaN이 적은 순으로 행 뽑아내기
MovieLens_pivot = pd.DataFrame(MovieLens_pivot, index=row_index.index)
# rating이 많은 사용자 1,000명
MovieLens_pivot = MovieLens_pivot.iloc[:1000,:]
col_index = MovieLens_pivot.isnull().sum(axis = 0).sort_values() # NaN이 적은 순으로 열 뽑아내기
MovieLens_pivot = pd.DataFrame(MovieLens_pivot, columns=col_index.index)
# rating이 많은 영화 1,000개
MovieLens_pivot = MovieLens_pivot.iloc[:,:1000]
# user_index = MovieLens_pivot.index.values # User ID 순서 기억하기
# user_index = pd.Series(user_index) # Index rename Simi인자로 넣기 위해 형태 변경
MovieLens_pivot.fillna(0, inplace=True)
MovieLens_pivot = MovieLens_pivot.astype('int64')
MovieLens_pivot = np.array(MovieLens_pivot) # 함수 인자로 넣기 위해 numpy array 형태로 변경

# 1. Cosine Similarity

In [108]:
def COS(data):
    simCOS = np.zeros(data.shape) # 0으로 초기화 된 행렬 생성'
    num_users = np.size(data, axis=0)
    for u in range(0, num_users): # 각 유저별로 for문 반복
        arridx_u = np.where(data[u, ] == 0)
        for v in range(u + 1, num_users):
            arridx_v = np.where(data[v, ] == 0)
            arridx = np.unique(np.concatenate((arridx_u, arridx_v), axis = None))
            
            U = np.delete(data[u, ], arridx)
            V = np.delete(data[v, ], arridx)
            
            if(np.linalg.norm(U) == 0 or np.linalg.norm(V) == 0):
                simCOS[u, v] = 0
            else:
                simCOS[u, v] = np.dot(U, V) / (np.linalg.norm(U) * np.linalg.norm(V))
            
            simCOS[v, u] = simCOS[u, v]
            
    return simCOS # 최종 행렬값 반환

# 2. Pearson Correlation Coefficient

In [109]:
def PCC(data):
    simPCC = np.zeros(data.shape) # 0으로 초기화 된 행렬 생성
    
    mean = np.nanmean(np.where(data != 0, data, np.nan), axis = 1)
    num_users = np.size(data, axis=0)
    for u in range(0, num_users):
        arridx_u = np.where(data[u, ] == 0)
        for v in range(u + 1, num_users):
            arridx_v = np.where(data[v, ] == 0)
            arridx = np.unique(np.concatenate((arridx_u, arridx_v), axis = None))
            
            U = np.delete(data[u, ], arridx) - mean[u]
            V = np.delete(data[v, ], arridx) - mean[v]
            
            if(np.linalg.norm(U) == 0 or np.linalg.norm(V) == 0):
                simPCC[u, v] = 0
            else:
                simPCC[u, v] = np.dot(U, V) / (np.linalg.norm(U) * np.linalg.norm(V))
            
            simPCC[v, u] = simPCC[u, v]
        
    return simPCC

In [111]:
COS(MovieLens_pivot)

array([[0.        , 0.94358936, 0.97035282, ..., 0.94719806, 0.95566258,
        0.98130691],
       [0.94358936, 0.        , 0.95506278, ..., 0.91679088, 0.93846546,
        0.97217696],
       [0.97035282, 0.95506278, 0.        , ..., 0.95078063, 0.97077589,
        0.98391051],
       ...,
       [0.94719806, 0.91679088, 0.95078063, ..., 0.        , 0.94613219,
        0.96066389],
       [0.95566258, 0.93846546, 0.97077589, ..., 0.94613219, 0.        ,
        0.9837906 ],
       [0.98130691, 0.97217696, 0.98391051, ..., 0.96066389, 0.9837906 ,
        0.        ]])

In [112]:
PCC(MovieLens_pivot)

array([[0.        , 0.21149714, 0.3543997 , ..., 0.316427  , 0.35636076,
        0.33395717],
       [0.21149714, 0.        , 0.14949384, ..., 0.16950372, 0.1423658 ,
        0.22727961],
       [0.3543997 , 0.14949384, 0.        , ..., 0.30757567, 0.35989088,
        0.28065287],
       ...,
       [0.316427  , 0.16950372, 0.30757567, ..., 0.        , 0.6863964 ,
        0.24508334],
       [0.35636076, 0.1423658 , 0.35989088, ..., 0.6863964 , 0.        ,
        0.18032181],
       [0.33395717, 0.22727961, 0.28065287, ..., 0.24508334, 0.18032181,
        0.        ]])

# Rating

In [15]:
A = np.array([[4, 1, 1, 4, 2, 3, 5, 0, 4, 0],
              [0, 4, 2, 0, 3, 2, 5, 0, 4, 3],
              [2, 0, 4, 5, 2, 0, 1, 3, 4, 2],
              [1, 4, 0, 1, 4, 5, 0, 3, 1, 2],
              [1, 2, 3, 4, 0, 3, 4, 4, 2, 5]])

In [80]:
# 임시로 만들어 놓기
def basic_mean(mat, sim, k):
    predicted_rating = np.zeros(mat.shape)
    
    mean = np.nanmean(np.where(mat != 0, mat, np.nan), axis=1)

    if(sim == 'COS'):
        Sim = COS(mat)
    elif(sim == 'PCC'):
        Sim = PCC(mat)
        
    k_neighbors = np.argsort(-Sim)
    k_neighbors = np.delete(k_neighbors, np.s_[k:], 1)

    NumUsers = np.size(mat, axis=0)
    
    for u in range(0, NumUsers):
        list_sim = Sim[u, k_neighbors[u]]
        list_rating = mat[k_neighbors[u]]
        list_mean = mean[k_neighbors[u]]
        
        mom = np.sum(list_sim)
        sun = np.sum(list_sim.reshape(-1,1) * (list_rating - list_mean.reshape(-1,1)), axis=0)
        predicted_rating[u] = mean[u] + sun / mom
        
    return predicted_rating

In [100]:
def basic_baseline(data, sim, k):
    # initializing (1000, 1000) numpy array with zeros 
    pred_rating = np.zeros(data.shape)

    # calculating means
    mean = np.nanmean(np.where(data != 0, data, np.nan)) # the mean of all ratings
    mean_u = np.nanmean(np.where(data != 0, data, np.nan), axis=1) # the mean of all users
    mean_i = np.nanmean(np.where(data != 0, data, np.nan), axis=0) # the mean of all items
    
    # base user, item
    b_u = mean_u - mean 
    b_i = mean_i - mean
    # b_ui = mean + b_u + b_i

    # selecting similarity fuction
    if sim == 'COS':
        Sim = COS(data)
    elif sim == 'PCC':
        Sim = PCC(data)

    # selecting top k users by sorting similarity array
    k_users = np.argsort(-Sim)
    k_users = np.delete(k_users, np.s_[k:], 1) # delete colomn under k
        
    # number of users with axis = 0 condition
    num_users = np.size(data, axis=0)
    num_items = np.size(data, axis=1)
    
    # calculating predicted ratings
    for u in range(0, num_users):
        list_sim = Sim[u, k_users[u]] # selecting top k similarity
        for i in range(0, num_items):
            list_rating = data[k_users[u], i].astype('float64') # k users' ratings on item i

            b_ui = mean + b_u[u] + b_i[i] # scalar
            b_vi = mean + b_u[k_users[u]] + b_i[i] # list
            
            # explanation of varialbles
            # mean_u[u] : user u의 평균
            # mean_i[i] : item i의 평균
            # b_u[u] : user u의 baseline
            # b_i[i] : item i의 baseline

            # calculation
            mom = np.sum(list_sim) #분모
            son = np.sum(list_sim * (list_rating - b_vi)) #분자
            pred_rating[u, i] = b_ui + son / mom
            
    return pred_rating

In [113]:
basic_baseline(MovieLens_pivot, 'COS', 2)

array([[ 1.81655320e+00,  1.81655320e+00,  1.81655320e+00, ...,
        -6.80315149e-01, -6.80315149e-01,  1.31717953e+00],
       [ 3.42179744e+00,  3.92253386e+00,  3.42179744e+00, ...,
        -5.76729721e-01,  1.91958819e+00, -5.76729721e-01],
       [ 4.58647989e+00,  5.08648408e+00,  5.08648408e+00, ...,
         8.64840756e-02,  8.64840756e-02,  2.08650080e+00],
       ...,
       [ 3.98392408e+00,  4.48403460e+00,  3.48381356e+00, ...,
         1.48447670e+00, -5.15965395e-01,  1.48359251e+00],
       [ 3.93045873e+00,  4.43023094e+00,  4.43023094e+00, ...,
        -5.69769058e-01, -5.69769058e-01,  3.43023094e+00],
       [ 5.00296743e+00,  2.50221309e+00,  2.50372176e+00, ...,
         2.96742717e-03,  2.00236396e+00,  2.96742717e-03]])

In [ ]:
basic_baseline(MovieLens_pivot, 'PCC', 2)

In [110]:
basic_baseline(A, 'COS', 2)

array([[ 0.28988047,  2.81978035,  2.28988047,  1.75998059,  1.32974698,
         2.28988047,  4.30981372,  1.75998059,  2.81978035,  3.77991384],
       [ 2.74888141,  1.72480939,  2.21879139,  4.2308274 ,  1.24286341,
         3.2308274 ,  4.7368454 ,  2.20675538,  3.24286341,  2.70073737],
       [ 0.18909227,  2.65715391,  2.18909227,  1.72103063,  1.14650779,
         2.18909227,  4.16780003,  1.72103063,  2.65715391,  3.69973839],
       [-0.08579346,  2.44582768,  1.91420654,  1.3825854 ,  0.95636806,
         1.91420654,  3.9352873 ,  1.3825854 ,  2.44582768,  3.40366616],
       [ 1.95132724,  2.47897423,  1.47107509,  1.95132724,  2.47107509,
         2.46317595,  4.96712552, -0.03287448,  3.96712552,  1.47897423]])